# Pharmacy Data Enrichment
This notebook is for exploring how to best enrich data collected from the state database for additional insights. 

## Notebook Configuration

In [2]:
import pandas as pd

# Set pandas to display more rows/columns for wider dataframes
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)     # Adjust rows
pd.set_option('display.width', 1000)       # Increase width for better visibility

## Load Pharmacy Data

In [3]:
pharmacies = pd.read_csv('attempt_19.csv')
pharmacies.head()

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,CEDARS-SINAI MEDICAL CENTER -4TH FLOOR MAIN
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,"July 1, 2025",ANAHEIM,California,ORANGE,92807.0,NaN
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,"March 1, 2025",SALINAS,California,MONTEREY,93901.0,NaN
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,"December 1, 2025",WARRENDALE,Pennsylvania,OUT OF STATE,15086.0,NaN
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,"December 1, 2025",ORLANDO,Florida,OUT OF STATE,32822.0,NaN
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,"July 1, 2025",ESCONDIDO,California,SAN DIEGO,92025.0,NaN


## Enrich Pharmacy Categorization

- [y/n] - is this a government facility?
- [y/n] - is this a satellite location?
- [y/n] - is this an infusion center? 
- [y/n] - is this a hospital or medical center? 
- [y/n] - is research institute or lab
- [y/n] - is this strictly a veterinarian pharmacy? 
- [y/n] - provides Vitamin/Nutrition Therapy drugs
- [y/n] - provide GLP1 drugs (Semaglutide, Tirzepatide)

Available Specialities:
- specialtyCancer
- specialtyHomeInfusion
- specialtyNutrition
- 

In [32]:
# Create an isGovernment column and populate it based on the License Type
pharmacies['isGovernment'] = pharmacies['License Type'].str.contains('Government', case=False, na=False)

# Create an isSatellite column and populate it based on the License Type
pharmacies['isSatellite'] = pharmacies['License Type'].str.contains('Satellite', case=False, na=False)

# Create an isInfusionCenter column and populate it based on the Pharmacy Name
pharmacies['isInfusionCenter'] = pharmacies['License Type'].str.contains('Infusion Center', case=False, na=False)

# List of keywords to check for in the 'License Type' column
keywords = [
    'HOSPITAL', 'MEDICAL CENTER', 'ADVENTIST HEALTH', 'MEMORIAL', 'CAMPUS', 
    'REGIONAL', 'MED CTR', 'KAISER', 'OF USC', 'REHABILITATION', 'CEDARS-SINAI', 'CITY OF HOPE', 'CANCER CENTER', 'HOSPTIAL', 'CLINIC', 'HSP','OUTPATIENT', 'HEALTH SERVICES', 'HEALTH CENTER', 'COUNTY ONCOLOGY', 'HEALTH SYSTEM', 'STANFORD HEALTH CARE', 'CENTER FOR', 'UC DAVIS', 'UCI HEALTH']

# Create a pattern that matches any of the keywords
keywords_pattern = '|'.join([re.escape(keyword) for keyword in keywords])

# Create 'isHospital' column based on whether any of the keywords are present in 'Pharmacy Name'
pharmacies['isHospital'] = pharmacies['Pharmacy Name'].str.contains(keywords_pattern, case=False, regex=True, na=False)









In [33]:
# Review Pharmacies that are NOT a hospital currently 
non_hospital_pharmacies = pharmacies[pharmacies['isHospital'] == False]
non_hospital_pharmacies

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,CEDARS-SINAI MEDICAL CENTER -4TH FLOOR MAIN,isGovernment,isSatellite,isInfusionCenter,isHospital
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,"July 1, 2025",ANAHEIM,California,ORANGE,92807.0,NaN,False,False,False,False
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,"March 1, 2025",SALINAS,California,MONTEREY,93901.0,NaN,False,False,False,False
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,"December 1, 2025",WARRENDALE,Pennsylvania,OUT OF STATE,15086.0,NaN,False,False,False,False
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,"December 1, 2025",ORLANDO,Florida,OUT OF STATE,32822.0,NaN,False,False,False,False
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,"July 1, 2025",ESCONDIDO,California,SAN DIEGO,92025.0,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,WEDGEWOOD PHARMACY LLC,NSC 101918,Nonresident Sterile Compounding Pharmacy,CLEAR,"April 1, 2025",SCOTTSDALE,Arizona,OUT OF STATE,85251.0,NaN,False,False,False,False
838,WELLNESS CENTER PHARMACY INC DBA DESIGNER DRUGS,NSC 100938,Nonresident Sterile Compounding Pharmacy,CLEAR,"November 1, 2025",CHATTANOOGA,Tennessee,OUT OF STATE,37421.0,NaN,False,False,False,False
839,WELLSCRIPT,LSC 100178,Sterile Compounding Pharmacy,CLEAR,"June 1, 2025",REDDING,California,SHASTA,96001.0,NaN,False,False,False,False
843,"WOMENS INTERNATIONAL PHARMACY, INC. DBA BELMAR...",NSC 101754,Nonresident Sterile Compounding Pharmacy,CLEAR,"March 1, 2025",YOUNGTOWN,Arizona,OUT OF STATE,85363.0,NaN,False,False,False,False
